In [1]:
from simulation_helpers import *
from heuristics import *
from optimal_policy import V
import pandas as pd

import time

### Optimal Policy vs Heuristic Tests
Code to compare the heuristics to the optimal policy on a few different problem instances

In [ ]:
graph_connectedness = [0.01, 0.1, 0.5, 1]
p_list = [0,0.3,0.7,1]
q_list = [0,0.3,0.7,1]
n_list = [2,3,5]
H = 9
M = 20
heuristics = [selectRandom, highRisk, highConnection, highBelief, sampleBelief, highBeliefRisk]
num_sim_heur = 5
num_sim_A = 3


results = pd.DataFrame()
for n in n_list:
    start1 = time.time()
    print('Running size: ', n)
    for adj_con in graph_connectedness:
        for i in range(num_sim_A):
            A = generateAdjacenyMatrix(n, adj_con)
            for p in p_list:
                for q in q_list:
                    for L in np.arange(n):
                        #Compute optimal Policy
                        #print('Running Optimal')
                        V_saved = {}
                        A_saved = {}
                        res_opt = []
                        for h in range(H+1):
                            res_opt.append(V(np.ones(n), np.ones(n)/n * M, h, False, p, q, L, A, V_saved, A_saved, M=M))
                        
                        result = pd.DataFrame({'population_n':n,
                                                'infected': res_opt,
                                                   'h': np.arange(H+1),
                                                   'algo': 'optimal',
                                                    'p': p,
                                                  'q': q,
                                                  'L': L,
                                                  'connectedness': adj_con,
                                                  'sim_id': i})
                        results = results.append(result)
                        
                        #Get simulated results for heuristic
                        for algo in heuristics:
                            #print('Running: ', algo.__name__)
                            heuristic_res = []
                            for j in range(num_sim_heur):
                                heuristic_res.append(sample(algo, 
                                                  np.ones(n).astype(np.bool), 
                                                  H, p, q, L, A, n))
                            result = pd.DataFrame({'population_n':n,
                                                    'infected': np.array(heuristic_res).mean(axis=0),
                                                       'h': np.arange(H+1),
                                                       'algo': algo.__name__,
                                                        'p': p,
                                                      'q': q,
                                                      'L': int(L*n),
                                                      'connectedness': adj_con,
                                                      'sim_id': i})
                            results = results.append(result)
            
    results.to_csv('opt_vs_heuristic_tests.csv', index = False)    
    print("total time taken this loop: ", time.time() - start1)


Running size:  2
total time taken this loop:  13.82506275177002
Running size:  3
total time taken this loop:  49.95334720611572
Running size:  5


### Heuristics (Large Samples)
Code to generate results for testing heuristics onlarger problem instances.

In [ ]:
graph_connectedness = [0.01, 0.1, 0.5, 1]
p_list = [0,0.3,0.7,1]
q_list = [0,0.3,0.7,1]
n_list = [5, 10, 100]
L_list = [0,0.1,0.5]
H = 9
heuristics = [selectRandom, highRisk, highConnection, highBelief, sampleBelief, highBeliefRisk]
num_sim = 5


results = pd.DataFrame()
for n in n_list:
    start2 = time.time()
    print('Running size: ', n)
    for adj_con in graph_connectedness:
        for i in range(num_sim):
            A = generateAdjacenyMatrix(n, adj_con)
            for p in p_list:
                for q in q_list:
                    for L in L_list:
                        for algo in heuristics:
                            sample_res = sample(algo, 
                                              np.ones(n).astype(np.bool), 
                                              H, p, q, int(L*n), A, n)
                            result = pd.DataFrame({'population_n':n,
                                                    'infected': sample_res,
                                                   'h': np.arange(H+1),
                                                   'algo': algo.__name__,
                                                    'p': p,
                                                  'q': q,
                                                  'L': int(L*n),
                                                  'connectedness': adj_con,
                                                  'sim_id': i})
                            #print(result)
                            results = results.append(result)
            
    results.to_csv('heuristic_tests_random_graph.csv', index = False)    
    print("total time taken this loop: ", time.time() - start2)